In [12]:
# Para este ejemplo usaremos el dataset de open images
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [13]:
ta = pd.read_csv('../01_ML-CV/Open_Images_Data/test-annotations-bbox.csv')
ti = pd.read_csv('../01_ML-CV/Open_Images_Data/test-images.csv')
va = pd.read_csv('../01_ML-CV/Open_Images_Data/validation-annotations-bbox.csv')
vi = pd.read_csv('../01_ML-CV/Open_Images_Data/validation-images.csv')

## Datasets Utilizados

### Test Annotations (ta)
**Descripción**: Contiene las anotaciones de las cajas delimitadoras (bounding boxes) para las imágenes de prueba.
**Uso**: Se utiliza para evaluar el rendimiento del modelo de detección y clasificación de objetos en un conjunto de datos no visto durante el entrenamiento.

### Test Images (ti)
**Descripción**: Contiene una lista de imágenes de prueba.
**Uso**: Se utiliza para obtener las imágenes que serán evaluadas con las anotaciones correspondientes en el dataset de prueba.

### Validation Annotations (va)
**Descripción**: Contiene las anotaciones de las cajas delimitadoras (bounding boxes) para las imágenes de validación.
**Uso**: Se utiliza para validar el modelo durante su desarrollo, permitiendo ajustar hiperparámetros y mejorar la precisión del modelo.

### Validation Images (vi)
**Descripción**: Contiene una lista de imágenes de validación.
**Uso**: Se utiliza para obtener las imágenes que serán validadas con las anotaciones correspondientes en el dataset de validación.


# Paso 1: Inspeccionar los Datos

In [14]:
# Información general del dataset de imágenes de prueba
print("Test Images (ti):")
print(ti.info())
#print(ti.head())

Test Images (ti):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125436 entries, 0 to 125435
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   ImageID             125436 non-null  object
 1   Subset              125436 non-null  object
 2   OriginalURL         125436 non-null  object
 3   OriginalLandingURL  125436 non-null  object
 4   License             125436 non-null  object
 5   AuthorProfileURL    125436 non-null  object
 6   Author              125436 non-null  object
 7   Title               125436 non-null  object
 8   OriginalSize        125436 non-null  int64 
 9   OriginalMD5         125436 non-null  object
 10  Thumbnail300KURL    120869 non-null  object
dtypes: int64(1), object(10)
memory usage: 10.5+ MB
None


In [15]:
# Información general del dataset de imágenes de prueba
print("Test Images (ta):")
print(ta.info())
#print(ta.head())

Test Images (ta):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625282 entries, 0 to 625281
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ImageID      625282 non-null  object 
 1   Source       625282 non-null  object 
 2   LabelName    625282 non-null  object 
 3   Confidence   625282 non-null  int64  
 4   XMin         625282 non-null  float64
 5   XMax         625282 non-null  float64
 6   YMin         625282 non-null  float64
 7   YMax         625282 non-null  float64
 8   IsOccluded   625282 non-null  int64  
 9   IsTruncated  625282 non-null  int64  
 10  IsGroupOf    625282 non-null  int64  
 11  IsDepiction  625282 non-null  int64  
 12  IsInside     625282 non-null  int64  
dtypes: float64(4), int64(6), object(3)
memory usage: 62.0+ MB
None


In [16]:
# Información general del dataset de imágenes de prueba
print("Test Images (vi):")
print(vi.info())
#print(vi.head())

Test Images (vi):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41620 entries, 0 to 41619
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ImageID             41620 non-null  object
 1   Subset              41620 non-null  object
 2   OriginalURL         41620 non-null  object
 3   OriginalLandingURL  41620 non-null  object
 4   License             41620 non-null  object
 5   AuthorProfileURL    41620 non-null  object
 6   Author              41620 non-null  object
 7   Title               41620 non-null  object
 8   OriginalSize        41620 non-null  int64 
 9   OriginalMD5         41620 non-null  object
 10  Thumbnail300KURL    40100 non-null  object
dtypes: int64(1), object(10)
memory usage: 3.5+ MB
None


In [17]:
# Información general del dataset de imágenes de prueba
print("Test Images (va):")
print(va.info())
#print(ti.head())

Test Images (va):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204621 entries, 0 to 204620
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ImageID      204621 non-null  object 
 1   Source       204621 non-null  object 
 2   LabelName    204621 non-null  object 
 3   Confidence   204621 non-null  int64  
 4   XMin         204621 non-null  float64
 5   XMax         204621 non-null  float64
 6   YMin         204621 non-null  float64
 7   YMax         204621 non-null  float64
 8   IsOccluded   204621 non-null  int64  
 9   IsTruncated  204621 non-null  int64  
 10  IsGroupOf    204621 non-null  int64  
 11  IsDepiction  204621 non-null  int64  
 12  IsInside     204621 non-null  int64  
dtypes: float64(4), int64(6), object(3)
memory usage: 20.3+ MB
None


In [18]:
# Tamaño
print("Shape:")
print(ti.shape)
print(ta.shape)
print(vi.shape)
print(va.shape)

Shape:
(125436, 11)
(625282, 13)
(41620, 11)
(204621, 13)


# Paso 2. Ajustar inconsistencias
- en este caso tenemos un buen volumen de datos sin valores nulos por lo que solo será necesario eliminar duplicados. Dependiendo la situación y el contexto de los datos, será la toma de la mejor decisión para ajustar. Puedes eliminar nulos, trabajar con medidas de tendencia central u otras para complementar la información, entore otras estrategias.

In [19]:
initial_shape_ta = ta.shape
ta = ta.drop_duplicates()
print(f"Duplicados eliminados en Test Annotations: {initial_shape_ta[0] - ta.shape[0]} filas eliminadas")

Duplicados eliminados en Test Annotations: 0 filas eliminadas


In [20]:
initial_shape_ti = ti.shape
ti = ti.drop_duplicates()
print(f"Duplicados eliminados en Test Imagess: {initial_shape_ti[0] - ti.shape[0]} filas eliminadas")

Duplicados eliminados en Test Imagess: 0 filas eliminadas


In [21]:
initial_shape_vi = vi.shape
vi = vi.drop_duplicates()
print(f"Duplicados eliminados en Validation Images: {initial_shape_vi[0] - vi.shape[0]} filas eliminadas")

Duplicados eliminados en Validation Images: 0 filas eliminadas


In [22]:
initial_shape_va = va.shape
va = va.drop_duplicates()
print(f"Duplicados eliminados en Validation Annotations: {initial_shape_va[0] - va.shape[0]} filas eliminadas")

Duplicados eliminados en Validation Annotations: 0 filas eliminadas


# Paso 3. Normalización de las Coordenadas de las Cajas Delimitadoras
- Para la normalización, utilizaremos MinMaxScaler de sklearn.preprocessing para escalar las coordenadas de las cajas delimitadoras
- - Dado que estamos trabajando con detección de objetos y los datasets contienen coordenadas de cajas delimitadoras (bounding boxes), es importante asegurarnos de que estas coordenadas estén en un rango consistente, como [0, 1].

In [23]:
scaler_ta = MinMaxScaler()
ta[['XMin', 'XMax', 'YMin', 'YMax']] = scaler_ta.fit_transform(ta[['XMin', 'XMax', 'YMin', 'YMax']])

scaler_va = MinMaxScaler()
va[['XMin', 'XMax', 'YMin', 'YMax']] = scaler_va.fit_transform(va[['XMin', 'XMax', 'YMin', 'YMax']])

In [24]:
print("Test Annotations (ta) después de la normalización:")
print(ta[['XMin', 'XMax', 'YMin', 'YMax']].describe())

print("Validation Annotations (va) después de la normalización:")
print(va[['XMin', 'XMax', 'YMin', 'YMax']].describe())

Test Annotations (ta) después de la normalización:
                XMin           XMax           YMin           YMax
count  625282.000000  625282.000000  625282.000000  625282.000000
mean        0.341017       0.660734       0.310061       0.668117
std         0.282472       0.281995       0.242598       0.273722
min         0.000000       0.000000       0.000000       0.000000
25%         0.077114       0.445438       0.102359       0.445661
50%         0.301111       0.700295       0.276068       0.703701
75%         0.556324       0.925448       0.481004       0.928981
max         1.000000       1.000000       1.000000       1.000000
Validation Annotations (va) después de la normalización:
                XMin           XMax           YMin           YMax
count  204621.000000  204621.000000  204621.000000  204621.000000
mean        0.340179       0.662969       0.308269       0.668580
std         0.282275       0.281285       0.242079       0.274952
min         0.000000       0.00000

# Paso 4. Verificar y Manejar Cualquier Otra Inconsistencia
- Dado que ya hemos cargado los datos, inspeccionado su estructura, eliminado duplicados y normalizado las coordenadas de las cajas delimitadoras, el siguiente paso es verificar y manejar cualquier otra inconsistencia que pueda existir en los datos. Esto puede incluir:
1. Verificar la coherencia de las etiquetas de clase: Asegurarse de que todas las etiquetas de clase sean válidas y consistentes.
2. Asegurar que todas las imágenes tengan anotaciones correspondientes: Asegurar que no haya imágenes sin anotaciones y viceversa.

In [25]:
# Verificar etiquetas de clase en Test Annotations
print("Etiquetas de clase en Test Annotations:")
print(ta['LabelName'].unique())

# Verificar etiquetas de clase en Validation Annotations
print("Etiquetas de clase en Validation Annotations:")
print(va['LabelName'].unique())

Etiquetas de clase en Test Annotations:
['/m/07j7r' '/m/015p6' '/m/05s2s' '/m/07yv9' '/m/0k4j' '/m/01g317'
 '/m/0283dt1' '/m/03q69' '/m/04hgtk' '/m/05r655' '/m/09j2d' '/m/0dzct'
 '/m/0dzf4' '/m/0c9ph5' '/m/0k0pj' '/m/0cgh4' '/m/018xm' '/m/09b5t'
 '/m/05zsy' '/m/0dv77' '/m/03120' '/m/01_bhs' '/m/02y6n' '/m/0270h'
 '/m/0284d' '/m/0cxn2' '/m/0138tl' '/m/083kb' '/m/02p0tk3' '/m/035r7c'
 '/m/04yx4' '/m/05y5lj' '/m/01d40f' '/m/01lcw4' '/m/03bt1vf' '/m/07r04'
 '/m/083wq' '/m/08dz3q' '/m/0d4v4' '/m/0h9mv' '/m/015h_t' '/m/02wbm'
 '/m/0bt9lr' '/m/03vt0' '/m/09kmb' '/m/01h3n' '/m/0cyhj_' '/m/04bcr3'
 '/m/0c_jw' '/m/02xwb' '/m/09kx5' '/m/01r546' '/m/05z55' '/m/01xq0k1'
 '/m/04rky' '/m/09j5n' '/m/0cnyhnx' '/m/04_sv' '/m/0zvk5' '/m/01prls'
 '/m/052lwg6' '/m/0l515' '/m/01xyhv' '/m/01mzpv' '/m/0k65p' '/m/02dl1y'
 '/m/0463sg' '/m/0kpqd' '/m/01jfm_' '/m/0199g' '/m/0bjyj5' '/m/014j1m'
 '/m/05n4y' '/m/06c54' '/m/014sv8' '/m/039xj_' '/m/019jd' '/m/07mhn'
 '/m/01xs3r' '/m/076bq' '/m/0ch_cf' '/m/04c0y' '/m/0

In [26]:
# Verificar que todas las imágenes en test tengan anotaciones correspondientes
test_image_ids = set(ti['ImageID'])
test_annotation_ids = set(ta['ImageID'])

# Imágenes sin anotaciones
test_images_without_annotations = test_image_ids - test_annotation_ids
print(f"Imágenes en Test sin anotaciones: {len(test_images_without_annotations)}")

# Verificar que todas las imágenes en validation tengan anotaciones correspondientes
validation_image_ids = set(vi['ImageID'])
validation_annotation_ids = set(va['ImageID'])

# Imágenes sin anotaciones
validation_images_without_annotations = validation_image_ids - validation_annotation_ids
print(f"Imágenes en Validation sin anotaciones: {len(validation_images_without_annotations)}")

Imágenes en Test sin anotaciones: 17277
Imágenes en Validation sin anotaciones: 5695


# Paso 5: Eliminar Imágenes sin Anotaciones
- Vamos a eliminar las imágenes sin anotaciones de los datasets de prueba y validación.

In [27]:
# Eliminar imágenes sin anotaciones del dataset de imágenes de prueba
if len(test_images_without_annotations) > 0:
    ti = ti[~ti['ImageID'].isin(test_images_without_annotations)]
    print(f"Eliminadas {len(test_images_without_annotations)} imágenes sin anotaciones del dataset de prueba.")

# Eliminar imágenes sin anotaciones del dataset de imágenes de validación
if len(validation_images_without_annotations) > 0:
    vi = vi[~vi['ImageID'].isin(validation_images_without_annotations)]
    print(f"Eliminadas {len(validation_images_without_annotations)} imágenes sin anotaciones del dataset de validación.")

Eliminadas 17277 imágenes sin anotaciones del dataset de prueba.
Eliminadas 5695 imágenes sin anotaciones del dataset de validación.


In [30]:
print(f"Nuevo tamaño del dataset de imágenes de prueba: {ti.shape}")
print(f"Nuevo tamaño del dataset de imágenes de validación: {vi.shape}")

Nuevo tamaño del dataset de imágenes de prueba: (108159, 11)
Nuevo tamaño del dataset de imágenes de validación: (35925, 11)


# Paso 6: Preparar los Datos para el Entrenamiento
- Ahora vamos a preparar los datos para el entrenamiento. Este paso incluye: Guardar los Datasets Preprocesados: Guardar los datasets preprocesados en archivos CSV para su uso posterior en el entrenamiento del modelo.

In [31]:
# Guardar los datasets preprocesados en archivos CSV

# Dataset de anotaciones de prueba
ta.to_csv('../01_ML-CV/data/preprocessed_test_annotations.csv', index=False)

# Dataset de imágenes de prueba
ti.to_csv('../01_ML-CV/data/preprocessed_test_images.csv', index=False)

# Dataset de anotaciones de validación
va.to_csv('../01_ML-CV/data/preprocessed_validation_annotations.csv', index=False)

# Dataset de imágenes de validación
vi.to_csv('../01_ML-CV/data/preprocessed_validation_images.csv', index=False)

print("Datasets preprocesados guardados correctamente.")

Datasets preprocesados guardados correctamente.


# Preprocesamiento de Datos para el Asistente Inteligente de Navegación

## Introducción

El preprocesamiento de datos es una etapa crucial en el desarrollo de modelos de Machine Learning. Este proceso implica preparar y transformar los datos en un formato adecuado para que los algoritmos de Machine Learning puedan utilizarlos eficazmente. La calidad de los datos y cómo se preparan pueden tener un impacto significativo en el rendimiento y la precisión de los modelos.

## Objetivos del Preprocesamiento

1. **Mejorar la Calidad de los Datos**: Datos ruidosos, inconsistentes o incompletos pueden afectar negativamente el rendimiento del modelo. El preprocesamiento ayuda a limpiar y preparar los datos, mejorando su calidad.
2. **Reducción de Sesgos**: Eliminar o corregir datos sesgados es crucial para evitar que el modelo aprenda patrones incorrectos.
3. **Optimización del Rendimiento del Modelo**: Transformar los datos a una escala común y eliminar características irrelevantes puede mejorar significativamente la eficiencia y la precisión del modelo.
4. **Manejo de Valores Faltantes**: Los valores faltantes pueden llevar a resultados incorrectos si no se manejan adecuadamente. El preprocesamiento permite abordar este problema de manera sistemática.

## Pasos de Preprocesamiento

### Paso 1: Cargar los Datasets

Cargamos los datasets utilizando `pandas` para poder trabajar con ellos en forma de dataframes.

### Paso 2: Inspeccionar los Datos

Inspeccionamos los datos para entender su estructura y contenido. Esto nos permite identificar posibles duplicados, valores faltantes e inconsistencias. Utilizamos funciones como `info()` para obtener información sobre el número de entradas, el tipo de datos y la cantidad de valores no nulos en cada columna, y `head()` para ver las primeras filas de cada dataset y entender mejor su contenido.

### Paso 3: Eliminar Duplicados

Eliminamos cualquier registro duplicado en los dataframes utilizando el método `drop_duplicates()`. Esto es crucial para asegurar que los datos no contengan redundancias que puedan sesgar los resultados del modelo.

### Paso 4: Normalización

Normalizamos las coordenadas de las cajas delimitadoras para asegurarnos de que estén en un rango consistente. Esto se hace utilizando `MinMaxScaler` para escalar las coordenadas de las cajas delimitadoras a un rango de [0, 1]. La normalización es importante para que los algoritmos de Machine Learning, especialmente aquellos sensibles a la escala de los datos, funcionen correctamente.

### Paso 5: Verificar Correspondencia entre Imágenes y Anotaciones

Verificamos que todas las imágenes tengan anotaciones correspondientes. Esto implica comprobar que cada imagen en los datasets de prueba y validación tenga al menos una anotación asociada. Identificamos y eliminamos cualquier imagen que no tenga anotaciones correspondientes para asegurar la coherencia de los datos.

### Paso 6: Guardar los Datasets Preprocesados

Guardamos los datasets preprocesados en archivos CSV para su uso posterior en el entrenamiento del modelo. Esto asegura que los datos estén listos y disponibles en un formato adecuado cuando procedamos con el entrenamiento del modelo de detección de objetos.

## Resumen

El preprocesamiento de datos es una etapa esencial en el pipeline de Machine Learning que asegura la calidad y la integridad de los datos utilizados para entrenar modelos. A través de la carga, inspección, eliminación de duplicados, normalización, verificación de correspondencia y guardado de los datasets, hemos preparado nuestros datos para el siguiente paso: el entrenamiento del modelo de detección de objetos. Estas técnicas aseguran que los datos estén en una forma óptima para el modelado, lo que lleva a modelos más precisos y robustos.